https://energiaeambiente.org.br/qual-a-diferenca-entre-gases-de-efeito-estufa-e-poluentes-atmosfericos-20220511#:~:text=Os%20principais%20GEE%20emitidos%20por,de%20combust%C3%ADveis%20ou%20pelo%20desmatamento.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os
import json
import statsmodels.stats.api as sms
from statsmodels.formula.api import ols
pd.set_option('display.max_columns', None)

## Procurando países com menos dados (menos cidades)

In [2]:
cities_data = []
with open('../data/air-quality/airquality-covid19-cities.json', encoding="utf-8") as json_file:
    cities_data = json.load(json_file)['data']

countries_repeated = []
for city in cities_data:
    countries_repeated.append(city["Place"]['country'])

countries = {}
for country in countries_repeated:
    countries[country] = countries_repeated.count(country)
    
final_countries = []
for country in countries:
    if(countries[country] >= 10):
        final_countries.append(country)

In [3]:
df_emissions = pd.read_csv('../data/Gas Emissions/total-ghg-emissions.csv',comment='#', low_memory=False)

df_emissions.columns = ['COUNTRY', 'CODE', 'DATE', 'EMISSIONS']

In [4]:
df_emissions

,COUNTRY,CODE,DATE,EMISSIONS
0,Afghanistan,AFG,1990,9.580000e+06
1,Afghanistan,AFG,1991,9.810000e+06
2,Afghanistan,AFG,1992,9.030000e+06
3,Afghanistan,AFG,1993,9.110000e+06
4,Afghanistan,AFG,1994,9.150000e+06
...,...,...,...,...
6144,Zimbabwe,ZWE,2015,4.230000e+07
6145,Zimbabwe,ZWE,2016,1.159200e+08
6146,Zimbabwe,ZWE,2017,1.155900e+08
6147,Zimbabwe,ZWE,2018,1.182200e+08


In [5]:
print(len(df_emissions.COUNTRY.unique()))
df_emissions.COUNTRY.unique()

205


array(['Afghanistan', 'Africa', 'Albania', 'Algeria', 'Andorra', 'Angola',
       'Antigua and Barbuda', 'Argentina', 'Armenia', 'Asia', 'Australia',
       'Austria', 'Azerbaijan', 'Bahamas', 'Bahrain', 'Bangladesh',
       'Barbados', 'Belarus', 'Belgium', 'Belize', 'Benin', 'Bhutan',
       'Bolivia', 'Bosnia and Herzegovina', 'Botswana', 'Brazil',
       'Brunei', 'Bulgaria', 'Burkina Faso', 'Burundi', 'Cambodia',
       'Cameroon', 'Canada', 'Cape Verde', 'Central African Republic',
       'Chad', 'Chile', 'China', 'Colombia', 'Comoros', 'Congo',
       'Cook Islands', 'Costa Rica', "Cote d'Ivoire", 'Croatia', 'Cuba',
       'Cyprus', 'Czechia', 'Democratic Republic of Congo', 'Denmark',
       'Djibouti', 'Dominica', 'Dominican Republic', 'Ecuador', 'Egypt',
       'El Salvador', 'Equatorial Guinea', 'Eritrea', 'Estonia',
       'Eswatini', 'Ethiopia', 'Europe', 'European Union (27)', 'Fiji',
       'Finland', 'France', 'Gabon', 'Gambia', 'Georgia', 'Germany',
       'Ghana', 'Gr

In [6]:
df_emissions.head(3)

,COUNTRY,CODE,DATE,EMISSIONS
0,Afghanistan,AFG,1990,9.580000e+06
1,Afghanistan,AFG,1991,9.810000e+06
2,Afghanistan,AFG,1992,9.030000e+06


In [7]:
df_iqar = pd.read_csv('../data/air-quality/results/iqar-countries-class-after.csv',comment='#', low_memory=False)

In [8]:
df_iqar

,DATE,CODE,CODE-2,MEAN,CLASSIFICATION
0,2014,AUS,AU,18.392635,0.166667
1,2014,AUT,AT,25.457333,0.400000
2,2014,BEL,BE,21.207407,0.333333
3,2014,BOL,BO,9.266667,0.000000
4,2014,BRA,BR,15.391667,0.166667
...,...,...,...,...,...
664,2022,UKR,UA,18.184177,0.500000
665,2022,USA,US,13.672793,0.166667
666,2022,UZB,UZ,88.309392,2.000000
667,2022,VNM,VN,19.482719,0.166667


In [9]:
df = pd.merge(df_iqar, df_emissions,  how='inner', on=['CODE','DATE'])
df.head(10)

,DATE,CODE,CODE-2,MEAN,CLASSIFICATION,COUNTRY,EMISSIONS
0,2014,AUS,AU,18.392635,0.166667,Australia,5.699800e+08
1,2014,AUT,AT,25.457333,0.400000,Austria,6.808000e+07
2,2014,BEL,BE,21.207407,0.333333,Belgium,1.041500e+08
3,2014,BOL,BO,9.266667,0.000000,Bolivia,1.167200e+08
4,2014,BRA,BR,15.391667,0.166667,Brazil,1.384990e+09
5,2014,CAN,CA,8.293134,0.000000,Canada,8.421400e+08
6,2014,CHE,CH,16.400000,0.000000,Switzerland,4.691000e+07
7,2014,CHL,CL,16.054090,0.166667,Chile,1.525000e+07
8,2014,CHN,CN,52.435908,1.166667,China,1.122848e+10
9,2014,COL,CO,14.338889,0.166667,Colombia,2.311500e+08


## Coverage

In [10]:
df_iqar.DATE.unique()

array([2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022], dtype=int64)

In [11]:
df_emissions.DATE.unique()

array([1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000,
       2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011,
       2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019], dtype=int64)

In [12]:
df.DATE.unique()

array([2014, 2015, 2016, 2017, 2018, 2019], dtype=int64)

In [13]:
df.tail(10)

,DATE,CODE,CODE-2,MEAN,CLASSIFICATION,COUNTRY,EMISSIONS
357,2019,THA,TH,21.660153,0.333333,Thailand,4.371800e+08
358,2019,TJK,TJ,127.767677,3.000000,Tajikistan,1.738000e+07
359,2019,TKM,TM,63.218935,2.000000,Turkmenistan,1.573300e+08
360,2019,TUR,TR,26.588440,0.666667,Turkey,4.598600e+08
361,2019,UGA,UG,133.613699,3.000000,Uganda,5.915000e+07
362,2019,UKR,UA,31.753694,0.500000,Ukraine,2.212900e+08
363,2019,USA,US,14.517852,0.166667,United States,5.771000e+09
364,2019,UZB,UZ,99.659306,2.000000,Uzbekistan,1.853900e+08
365,2019,VNM,VN,28.892669,0.500000,Vietnam,4.381100e+08
366,2019,ZAF,ZA,18.843618,0.166667,South Africa,5.621900e+08


In [14]:
df.CLASSIFICATION.unique()

array([0.16666667, 0.4       , 0.33333333, 0.        , 1.16666667,
       0.83333333, 1.33333333, 1.        , 0.2       , 2.        ,
       0.8       , 0.66666667, 0.25      , 0.5       , 3.        ,
       1.5       , 0.6       ])

## Removendo países com menos dados (menos cidades)

In [15]:
print('Existem {} países, porém vamos manter apenas {}'.format(len(df['CODE-2'].unique()), len(final_countries)))

Existem 84 países, porém vamos manter apenas 21


In [16]:
df = df[df['CODE-2'].isin(final_countries)]

In [17]:
#df = df[df['CODE-2'].isin(['US', 'CN', 'KR'])]

In [18]:
# conferindo
len(df['CODE-2'].unique())

20

In [19]:
print(list(df['COUNTRY'].unique()))

['Australia', 'Canada', 'Chile', 'China', 'Germany', 'Spain', 'France', 'United Kingdom', 'India', 'Italy', 'Japan', 'South Korea', 'Mexico', 'Netherlands', 'Poland', 'Turkey', 'United States', 'South Africa', 'Iran', 'Romania']


## Modelando com base nas emissões

In [20]:
from sklearn.preprocessing import MinMaxScaler

df_sklearn = df.copy()
  
df_sklearn['CLASSIFICATION'] = MinMaxScaler().fit_transform(np.array(df_sklearn['CLASSIFICATION']).reshape(-1,1))
df_sklearn['EMISSIONS'] = MinMaxScaler().fit_transform(np.array(df_sklearn['EMISSIONS']).reshape(-1,1))

display(df_sklearn)

,DATE,CODE,CODE-2,MEAN,CLASSIFICATION,COUNTRY,EMISSIONS
0,2014,AUS,AU,18.392635,0.125,Australia,0.061623
5,2014,CAN,CA,8.293134,0.000,Canada,0.083859
7,2014,CHL,CL,16.054090,0.125,Chile,0.016301
8,2014,CHN,CN,52.435908,0.875,China,0.932439
12,2014,DEU,DE,16.264619,0.125,Germany,0.081230
...,...,...,...,...,...,...,...
347,2019,POL,PL,19.823809,0.250,Poland,0.041218
349,2019,ROU,RO,12.022287,0.000,Romania,0.021457
360,2019,TUR,TR,26.588440,0.500,Turkey,0.052626
363,2019,USA,US,14.517852,0.125,United States,0.486554


In [21]:
import statsmodels.formula.api as smf
model = smf.ols("CLASSIFICATION ~ EMISSIONS", data=df_sklearn)
response = model.fit()
response.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:         CLASSIFICATION   R-squared:                       0.112
Model:                            OLS   Adj. R-squared:                  0.104
Method:                 Least Squares   F-statistic:                     14.56
Date:                Fri, 16 Dec 2022   Prob (F-statistic):           0.000219
Time:                        15:37:38   Log-Likelihood:                 20.529
No. Observations:                 118   AIC:                            -37.06
Df Residuals:                     116   BIC:                            -31.52
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.2037      0.022      9.154      0.000       0.160       0.248
EMISSIONS      0.3329      0.087      3.816      0.000       0.160       0.506
==============================================================================
Omnibus:                       45.337   Durbin-Watson:                   1.589
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               91.887
Skew:                           1.641   Prob(JB):                     1.11e-20
Kurtosis:                       5.814   Cond. No.                         4.71
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""